# Jupyter Notebook UI to graph your TAP data!

Version 1.5 - Joseph Liang, Rankin Lab
Updated:
1. Upgraded folder path selection application
2. Upgraded dataset management (less moving parts for end-user)
3. output changed from tif -> png
4. Added tap-correction function for missing taps (Still in progress - taken out for now)
5. Added plate column to evaluate plate-discrepancies

## Known bug: Step 2 an empty windows displays in Mac. May also apply to linux/windows.

## Beginner Essentials:
1. Shift-Enter to run each cell. After you run, you should see an output "done step #". If not, an error has occured
2. When inputting your own code/revising the code, make sure you close all your quotation marks '' and brackets (), [], {}.
3. Don't leave any commas (,) hanging! (make sure an object always follows a comma. If there is nothing after a comma, remove the comma!
4. Learning to code? Each line of code is annotated to help you understand how this code works!

## 3. Run all cells/steps sequentially, even the ones that do not need input

## Steps that require input: #3, #6.1, #7

# 1. Importing Packages Required (No input required, just run)

In [1]:
import pandas as pd #<- package used to import and organize data
import numpy as np #<- package used to import and organize data
import os #<- package used to work with system file paths
import seaborn as sns #<- package used to plot graphs
from matplotlib import pyplot as plt #<- another package used to plot graphs
from itertools import cycle #<- package used to iterate down rows (used in step 5 to add tap column)
import ipywidgets as widgets #<- widget tool to generate button and tab for graphs
from IPython.display import display #<- displays widgets
from tkinter import Tk, filedialog #<- Tkinter is a GUI package
print("done step 1")

done step 1


# 2. Pick filepath (just run and click button from output)

Run the following cell and click the button 'Select Folder' to pick a filepath.

## Important: Later on, this script uses the total file path for each file to import and group data. That means if your folder has whatever your strain is named, the script will not work.

(ex. if your folder has "N2" in it this script sees all files inside this folder as having the "N2" search key)

## An easy fix is to just rename your folder to something else (make your strains lower-case, or just have the date)

In [ ]:
### Select Folder App - After you run, you can select your folder for filepath
button = widgets.Button(description = 'Select Folder') #<- Creates a button variable
display(button) #<- displays this button on output
def select_folder(b): #<- This is an action. Requires a variable, so I put in an arbitrary one 'b'
    global folder_path #<- sets a variable as a global variable, not just within this action
    Tk().withdraw() #<- Tkinter likes to create annoying empty windows. This removes them
    folder_path = filedialog.askdirectory() #<- Opens up a file explorer window, and determines folder path
    Tk().update()#<- below
    Tk().destroy()#<- this and the line above it removes the file explorer window after a selection is made
    print(folder_path) #<- this helps confirm that this action was performed
    print('done step 2')
button.on_click(select_folder) #<- describes what happens when you click on this button

# 3. User Defined Variables (Add input here)

Here, we add some constants to help you blaze through this code.

3.1: Number of taps is pretty self-explanatory. How any taps does your experiment have? put in that number + 1 (N+1)!

This may be a bit confusing, but this is due to some coding syntax that you don't have to worry about.

3.2: Change your ISI number. This will be reflected in the name/title of the output figure.

In [2]:

number_of_taps = 30 ###<------ Taps in your experiment.

number_taps = range(1, number_of_taps + 1)  #<- do not change this


# if you have different ISIs in the same folder, then come back and change this 
# when you are graphing for the second set of data with the other ISI 
# (Generally data from same ISIs are graphed together)
# If changing ISI mid-analysis, you can just skip straight to step 8 after running this cell again

ISI = 10  ### <- What is your ISI? change accordingly

# Here, open up one of the trv files to determine the times for each of these taps. 
#lower = np.arange(99, 399, ISI) #(first tap, last tap+10s, ISI)
#upper = np.arange(101, 401, ISI) #(first tap, last tap+10s, ISI)
#tolerances = list(zip(lower, upper)) 
#taps = [i for i in range(1,31)]

# assign each tolerance to a tap number

#accurate_taps = list((zip(taps, tolerances)))

#for pair in accurate_taps:
#    tap = pair[0]
#    tolerance = pair[1]
#    print("tap "+str(tap), "tolerance="+str(tolerance))

print("done step 3")

done step 3


# 4. Constructing Filelist From Source File/Select File (Just run)

In [3]:
folder_path = '/Users/Joseph/Desktop/AVR14_10sISI' #- manual folder path if Tkinter is acting up

os.chdir(folder_path) #<- setting your working directory so that your images will be saved here

filelist = list() #<- empty list
for root, dirs, files in os.walk(folder_path): #<- this for loop goes through your folder 
    for name in files:
        if name.endswith('.trv'): #<- and takes out all files with a .trv (file that contains your data)
            filepath = os.path.join(root, name) #<- Notes down the file path of each data file
            filelist.append(filepath) #<- saves it into the list
    
print(filelist)
print('done step 4')

['/Users/Joseph/Desktop/AVR14_10sISI/N2_5x4_f94h20c_100s30x10s10s_C0706aa.trv', '/Users/Joseph/Desktop/AVR14_10sISI/AVR14_10sISI_TapHab_0706_2019/pMEC7_C1/20190706_154901/MEC7C1_5x4_f94h20c_100s30x10s10s_C0706ga.trv', '/Users/Joseph/Desktop/AVR14_10sISI/AVR14_10sISI_TapHab_0706_2019/pMEC7_C1/20190706_162142/MEC7C1_5x4_f94h20C_100s30x10s10s_A0706ga.trv', '/Users/Joseph/Desktop/AVR14_10sISI/AVR14_10sISI_TapHab_0706_2019/pAVR14_C4/20190706_151355/AVR14C4_5x4_f94h20C_100s30x10s10s_A0706ba.trv', '/Users/Joseph/Desktop/AVR14_10sISI/AVR14_10sISI_TapHab_0706_2019/pMEC7_C5/20190706_160037/MEC7C5_5x4_f94h20C_100s30x10s10s_B0706hc.trv', '/Users/Joseph/Desktop/AVR14_10sISI/AVR14_10sISI_TapHab_0706_2019/pMEC7_C5/20190706_150445/MEC7C5_5x4_f94h20c_100s30x10s10s_C0706ha.trv', '/Users/Joseph/Desktop/AVR14_10sISI/AVR14_10sISI_TapHab_0706_2019/pMEC7_C5/20190706_155800/MEC7C5_5x4_f94h20C_100s30x10s10s_A0706hb.trv', '/Users/Joseph/Desktop/AVR14_10sISI/AVR14_10sISI_TapHab_0706_2019/pMEC7_C2/20190706_161817

# 5. Process Data Function (Just Run)

In [15]:
def ProcessData(strain): #<- an example of a user-defined function
    strain_filelist = [x for x in filelist if strain in x] #<- goes through the list and filters for keyword
    Strain_N = len(strain_filelist) #<- N per strain, or number of plates
    Plate_N = 1
    print(f'this strain/treatment has {Strain_N} plates') #<- will output as the first number
    if Strain_N == 0:
        raise AssertionError ('{} is not a good identifier'.format(strain))
    else:
        pass
#     for f in strain_filelist:
#         DF_Total = pd.concat(pd.read_csv(f, sep=' ', skiprows = 4, header = None))
        DF_Total = pd.concat([pd.read_csv(f, sep=' ', skiprows = 4, header = None) for f in strain_filelist],
                      ignore_index=True) #<- imports your data files
        DF_Total = DF_Total.dropna(axis = 1) #<- cleans your data
        DF_Total = DF_Total.rename( #<- more cleaning
                    {0:'time',
                    2:'rev_before',
                    3:'no_rev',
                    4:'stim_rev',
                    7:'dist',
                    8:'dist_std',
                    9:'dist_stderr',
                    11:'dist_0th',
                    12:'dist_1st',
                    13:'dist_2nd',
                    14:'dist_3rd',
                    15:'dist_100th',
                    18:'dura',
                    19:'dura_std',
                    20:'dura_stderr',
                    22:'dura_0th',
                    23:'dura_1st',
                    24:'dura_2nd',
                    25:'dura_3rd',
                    26:'dura_100th'}, axis=1)
        #check function here for NaN Columns
        DF_Total['prob'] = DF_Total['stim_rev']/ (DF_Total['no_rev'] + DF_Total['stim_rev']) #<- calculate prob
        DF_Total['speed'] = DF_Total['dist']/DF_Total['dura'] #<- calculate speed
        DF_Total["taps"] = np.nan #<- add a column called taps, the x axis for your graphs
        taps = cycle(number_taps) 
        DF_Total['taps'] = [next(taps) for count in range(DF_Total.shape[0])]
        DF_Total_rows = int(DF_Total.shape[0])
        print(f'this strain/treatment has {DF_Total_rows} total taps') #<- Outputs as the second number. Check if you are missing taps!
        DF_Final = DF_Total[["time", "taps", "dura", "dist", "prob", "speed"]].copy()
#         DF_Final = missing_taps(DF_Total[["time", "taps", "dura", "dist", "prob", "speed"]].copy(), accurate_taps, tolerances) #<- Finally, clean data!
#     if type(DF_Total_rows/number_of_taps) == int:
#         pass
#     else:
#         raise ValueError ("{} is missing taps in your data!".format(strain))
    return{
            'N': Strain_N,
            'Confirm':DF_Total,
            'Final': DF_Final}
    print('done!')

# def missing_taps(df, accurate_taps, tolerances):
#     """This functions checks a dataframe for missing taps and renumbers the tap
#     column accordingly.
    
#     :param df: dataframe with columns time, tap, speed
#     :type: pandas.core.frame.DataFrame
#     :param accurate_taps: list  of tuples (tuple[0] is the tap number and tuple[1] is a tuple of the tolerance range)
#     :type: list
#     :param tolerances: list of tuples of the tolerance ranges
#     :type: list
    
#     :return: dataframe with accurate tap numbers
#     :type: pandas.core.frame.DataFrame
#     """
#     for time, tol in zip(df["time"], tolerances):
#         lower_limit = tol[0]
#         upper_limit = tol[1]
#         print(lower_limit, time, upper_limit)
#         if time >= lower_limit and time <= upper_limit:
#             pass
#         else:
#             for pair in tolerances:
#                 lower_limit = pair[0]
#                 upper_limit = pair[1]
#                 # find the tolerance that should correspond to this time
#                 if time >= lower_limit and time <= upper_limit:
#                     # find index of accurate tolerance
#                     index = tolerances.index(pair)
#                     # find the corresponding tap number to the correct tolerance
#                     correct_tap_num = accurate_taps[index][0]
#                     # find the index of the time value with the wrong tap number
#                     index_wrong_tap = df.time[df.time == time].index.values.astype(int)[0]
#                     wrong_tap_num = df.taps[index_wrong_tap]
#                     print("Changing tap {} to tap {}".format(wrong_tap_num, correct_tap_num))
#                     # replace the wrong tap number with the correct value
#                     df.loc[index_wrong_tap, "taps"] = correct_tap_num
#                     break
#     print('finish working on fixing taps')

    
def insert_plates(df):   
    """This function inserts a plate column into a dataframe.
    
    :param df: any dataframe
    :type: pandas.core.frame.DataFrame
    
    :return: dataframe with a plate column
    :type: pandas.core.frame.DataFrame
    """
    plate = 0 

    # initialize plate column with nans
    df["plate"] = ""

    # create array of start and end indices for each cycle of 30 taps. 

    for index, row in df.iterrows():
        if len(df)-1 == index:
            df["plate"][index] = plate
            print('yes')
            break
        if df["taps"][index] == 1:
            plate += 1
            df["plate"][index] = plate
        else:
            df["plate"][index] = plate

print('done step 5')

done step 5


# 6.1 Process Data (PLEASE READ, Add input here)

Here is the hardest part - From your naming convention, you need to pick a unique identifier for each group.

This means that all of names of your files for that strain should have that in common but is not commone with across all other files! If you did a good job naming your files and following a good naming convention, this should be easy.

## Be careful and really look hard in your naming structure. Note you want an unique identifier in the entire file path for the same group of files. An easy mistake is to have the strain name in the overall folder name, in this case if you use your strain name as a keyword it would include all files in that folder!

For example, if all your N2 files have a certain pattern like "N2_5x4" in this following example:
'/Users/Joseph/Desktop/AVR14_10sISI_TapHab_0710_2019/N2/20190710_141740/N2_5x4_f94h20c_100s30x10s10s_C0710ab.trv'

Then you need to set that identifier for the strain keyword:
'Strain_1' = 'N2_5x4'

## Depending on how many strains you are running for comparison, you may need to add/delete some lines!

## You are not naming your data groups here, we have a step for that later!
## Here, you want to note down ALL the strains you have in the folder

If you have just 2 strains, add hashtags (#) in front of the lines you do not need.
If you need more strains, just add more lines, following the same format!

In [5]:
#Format: 'Strain_#'' = 'unique_identifier'

### Make into dictionary
StrainNames = {
    'Strain_1' : 'N2',  #<- each strain will be designated to a unique identifier here
    'Strain_2' : 'MEC7C5',
    'Strain_3' : 'MEC7C1',
    'Strain_4' : 'MEC7C2',
    'Strain_5' : 'MEC7A1',
    'Strain_6' : '',
    'Strain_7' : '', #<- empty entries are for those hardcore trackers that tracks this many strains
    'Strain_8' : '',
    'Strain_9' : '',
    'Strain_10' : '',
    'Strain_11' : '',
    'Strain_12' : '',
    'Strain_13' : '',
    'Strain_14' : '',
    'Strain_15' : '',}
#...etc, etc

print('done step 6.1')

done step 6.1


# 6.2 Process Data (just run this cell)

In [18]:
DataLists = [0]  #<- generates empty list. 0 is there to account for python's index starting at 0. 
# we want indexing to start at 1 (when I say #1 I want the first point, not the second point)

for s in StrainNames.values(): #<- goes through the dictionary in step 6.1 and processes data
    if not s == '':
        DataLists.append(ProcessData(s)['Final']) #<- appends all data into a list of dataframes


#missing_taps(DataLists[1][90:120], accurate_taps, tolerances)
#for df in DataLists[1:]:
#    missing_taps(df, accurate_taps, tolerances)
for df in DataLists[1:]: 
   insert_plates(df)
# x = insert_plates(DataLists[1]) #something is wrong with mutating DataFrame here 
# print(x)
#print(DataLists[1])
#print(len(DataLists))        
print('done step 6.2')

this strain/treatment has 7 plates
this strain/treatment has 208 total taps
this strain/treatment has 6 plates
this strain/treatment has 180 total taps
this strain/treatment has 5 plates
this strain/treatment has 150 total taps
this strain/treatment has 5 plates
this strain/treatment has 150 total taps
this strain/treatment has 3 plates
this strain/treatment has 90 total taps
yes
yes
yes
yes
yes
done step 6.2


/Users/Joseph/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Joseph/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Joseph/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [19]:
print(DataLists[1][90:120])

        time  taps  dura   dist      prob     speed plate
90    99.983     1  0.00  0.000  0.000000       NaN     4
91   109.982     2  2.23  0.767  0.750000  0.343946     4
92   119.977     3  2.16  0.739  0.815789  0.342130     4
93   129.992     4  1.70  0.527  0.688889  0.310000     4
94   139.968     5  1.63  0.532  0.711111  0.326380     4
95   149.967     6  2.22  0.720  0.490196  0.324324     4
96   159.983     7  1.76  0.564  0.517241  0.320455     4
97   169.942     8  1.52  0.472  0.600000  0.310526     4
98   179.952     9  1.80  0.533  0.592593  0.296111     4
99   189.973    10  1.77  0.536  0.545455  0.302825     4
100  199.944    11  2.02  0.619  0.357143  0.306436     4
101  209.941    12  2.02  0.630  0.488889  0.311881     4
102  229.994    13  1.62  0.511  0.545455  0.315432     4
103  239.946    14  1.38  0.362  0.510638  0.262319     4
104  249.945    15  1.65  0.495  0.368421  0.300000     4
105  259.982    16  1.57  0.447  0.474576  0.284713     4
106  269.998  

# 7. Grouping Data and Naming (Optional: Add input here)

Here, you get to name your data groups/strain! Name your groups however you like under between the quotation marks for each strain.

For example: If your Strain1 is N2 and you wish for the group to be called N2,
your line should look like:

DataLists[x].assign(dataset = 'N2')

## Go back to step 6.1 to check which strain is which item on the DataLists.
In this example, the first item on DataLists is N2.


## Remember: Put your name in quotes. (ex: 'N2' and not N2)

As default, the names are set to the unique identifier labels.

## Depending on the number of strains you are running the comparison, you may have to delete/add lines of code (following the same format). 
## Remember to add/delete commas too.

# If you want to change your groups, you do that here. 
For example, if you have 5 strains in your folder but only want to compare between 2 or 3 strains, designate that here and follow through with steps 6 and 7. Once you are done, come back to step 6 and change your groups again (You are going to have to change your graph titles for the second run-through though)!

In [ ]:
TotalConcatenated = pd.concat([ #<- this function joins your data together with an extra column for which group 
                          DataLists[1].assign(dataset = StrainNames.get("Strain_1")), 
                          DataLists[2].assign(dataset = StrainNames.get("Strain_2")),
                          DataLists[3].assign(dataset = StrainNames.get("Strain_3")),
                          DataLists[4].assign(dataset = StrainNames.get("Strain_4")),
                          DataLists[5].assign(dataset = StrainNames.get("Strain_5")),
                          
])
# ...etc etc
print(TotalConcatenated)
print('done step 7')

# 8. Graph Data (Probability, Distance, Duration, Speed of Tap Habituation)

## Here, feel free to change the graph title and axis labels! Images are automatically saved in your folder with your data!

Note: It has been agreed by the lab that distance is quite an outdated measure, and the three top measures we consider are:

## probability, duration and speed.

In [ ]:
out1 = widgets.Output() #<- this is part of the code that helps display your graphs in tabs
out2 = widgets.Output()
out3 = widgets.Output()
out4 = widgets.Output()

tab = widgets.Tab(children = [out1, out2, out3, out4]) #<- more code to support tab display
tab.set_title(0, 'Probability')
tab.set_title(1, 'Distance')
tab.set_title(2, 'Duration')
tab.set_title(3, 'Speed')
display(tab)

plt.rcParams.update({'lines.markeredgewidth': 1})

# Probability
with out1:
    plt.figure(linewidth = 2.5) #<- Make your figure lines THICCCC for clean display (posters)
    plt.figure(figsize=(12,10)) #<- set your figure size (width, length)
    plt.gca().xaxis.grid(False) #<- gets rid of x-axis markers to make data look clean 
    ax = sns.pointplot(x="taps", #<- Here we use seaborn as our graphing package. 
                 y="prob",  
                 data = TotalConcatenated,
                 hue = 'dataset', #<- Here we use the extra column from step 6 to separate by group
                 palette = 'deep', #<- Change colour palette here if you like
                 ci = 95) #<- Confidence interval. 95 = standard error
    plt.xlabel("Taps") #<- X-axis title
    plt.ylabel("Probability") #<- Y-Axis title
    plt.title(f"Probability of Tap Habituation, {ISI}ISI") #<- Figure Title
    ax.legend(loc = 'upper right', fontsize = '12') #<- location of your legend
    plt.savefig(f'Probability_{ISI}ISI.png', format='png', dpi=900) #<- saves your file to your folder at certain DPI
    plt.show()

# Distance
with out2:
    plt.figure(linewidth = 2.5)
    plt.figure(figsize=(12,10))
    plt.gca().xaxis.grid(False)
    ax = sns.pointplot(x="taps", 
                 y="dist", 
                 data = TotalConcatenated,
                 hue = 'dataset',
                 palette = 'deep',
                 ci = 95)
    plt.xlabel("Taps", fontsize = '12')
    plt.ylabel("Distance", fontsize = '12')
    plt.title(f"Distance of Tap Habituation, {ISI}ISI", fontsize = '16')
    ax.legend(loc = 'upper right', fontsize = '12')
    plt.savefig(f'Distance_{ISI}ISI.png', format='png', dpi=900)
    plt.show()

# Duration
with out3:
    plt.figure(linewidth = 2.5)
    plt.figure(figsize=(12,10))
    plt.gca().xaxis.grid(False)
    ax = sns.pointplot(x="taps", 
                 y="dura", 
                 data = TotalConcatenated,
                 hue = 'dataset',
                 palette = 'deep',
                 ci = 95)
    plt.xlabel("Taps", fontsize = '12')
    plt.ylabel("Duration", fontsize = '12')
    plt.title(f"Duration of Tap Habituation, {ISI}ISI", fontsize = '16')
    ax.legend(loc = 'upper right', fontsize = '12')
    plt.savefig(f'Duration_{ISI}ISI.png', format='png', dpi=900)
    plt.show()

# Speed
with out4:
    plt.figure(linewidth = 2.5)
    plt.figure(figsize=(12,10))
    plt.gca().xaxis.grid(False)
    ax = sns.pointplot(x="taps", 
                 y="speed", 
                 data = TotalConcatenated,
                 hue = 'dataset',
                 palette = 'deep',
                 ci = 95)
    plt.xlabel("Taps", fontsize = '12')
    plt.ylabel("Speed", fontsize = '12')
    plt.title(f"Speed of Tap Habituation, {ISI}ISI", fontsize = '16')
    ax.legend(loc = 'upper right', fontsize = '12')
    plt.savefig(f'Speed_{ISI}ISI.png', format='png', dpi=900)
    plt.show()

# Done!